In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import pprint
import csv
import datetime
from datetime import datetime
import nltk
import requests
import csv
import json
import pickle
import time
from tqdm import tqdm

# nltk.download('punkt')

In [5]:
def load_list_from_file(filename):
    with open(filename, 'rb') as file:
        data_list = pickle.load(file)
    return data_list

APArticlesList = list(load_list_from_file("2023519-189 - APNews"))

In [7]:
APArticlesList

[['More News',
  'https://apnews.com/article/video-cop-hood-chase-iowa-arrest-cbdc638ee621f08d1f6e2b99997e7523'],
 ['Australian police use Taser on 95-year-old with dementia who held steak knife',
  'https://apnews.com/article/taser-95-year-old-woman-australia-police-af221fd0e36e508ad8d25409ebbd8337'],
 ["Michigan boy who used slingshot to save sister says he 'was just lucky'",
  'https://apnews.com/article/boy-slingshot-saves-sister-e1617c360273d305d18025fb6e4b9383'],
 ['British climber scales Everest for 17th time, the most by a non-Sherpa guide',
  'https://apnews.com/article/everest-british-climber-new-record-7ccfb9fcebc08771ae45a4446d420da1'],
 ["Dave Matthews Band has new album 'Walk Around the Moon' and perspective: 'Everything is kind of new'",
  'https://apnews.com/article/dave-matthews-band-moon-tour-964072bbcda893ad11c28e6fb048aade'],
 ['Australian police use Taser on 95-year-old with dementia who held steak knife',
  'https://apnews.com/article/taser-95-year-old-woman-austr

In [51]:
current_time = datetime.now().strftime("%Y%m%d_%H%M%S") 
fileformat = ".txt"
filename = current_time + "-MasterListOfArticles"
filename = filename + fileformat 
print(filename)

20230520_132139-MasterListOfArticles.txt


NameError: name 'current_file' is not defined

In [20]:
def save_list_to_file(data_list, filename):
    with open(filename, 'wb') as file:
        pickle.dump(data_list, file)

In [52]:
# All article dictionaries will be stored here

master_list_of_articles = []
JSONifiedArticleList = []

def article_info_function(URL):
    
    article_information = {}
    
    # Plug the parameter into 
    response = requests.get(URL)
    APArticle = BeautifulSoup(response.content, 'html.parser')
    
    # Find the canonical link
    canonical_links = APArticle.find_all('link', rel='canonical')
    # print(canonical_links)

    # Print the href attribute of each matching link
    for link in canonical_links:
        self_URL = link.get('href')
        # print(self_URL)

    article_information["self_URL"] = self_URL
    
    # Find the Headline of the article
    main_Headline = APArticle.find_all('h1')
    main_Headline = main_Headline[0].text
    article_information["headline"] = main_Headline
    
    author_list = []

    # Find who wrote the article
    scripts = APArticle.find('script', attrs={'data-rh': 'true', 'type': 'application/ld+json'})
    JSONscript = json.loads(scripts.text)
    author_list = JSONscript['author']

    article_information["author(s)"] = author_list
    
    # Get Image Information
    image_data = []
    try:
        image_url = JSONscript['image']
        image_caption_div = APArticle.find_all('div', attrs={'data-key': 'embed-caption'})
    except:
        image_url = ""
        image_caption_div = ""
    try:
        image_caption = image_caption_div[0].text
    except:
        image_caption = ""
    try:
        image_attribution = image_caption[-50:].replace(")", "").split("(")[1]
    except:
        image_attribution = image_caption[-20:]

    image_data.append([image_url, image_caption, image_attribution])
    
    # Get paragraph content
    article_content = APArticle.find_all('div', {'class': 'Article', 'data-key': 'article'})

    paragraphCount = 0
    try: 
        inner_para = article_content[0].find_all('p')

        article_paragraphs = []

        for index, para in enumerate(inner_para):

            paragraphCount += 1
            inner_text = para.text 
            inner_links = para.find_all('a')
            content, href = "", ""

            # reset this array
            inner_link = []

            # Traverse the list to get the links and the associated text    
            for link in inner_links:
                content = link.string.strip()
                href = link['href']
                if href[0] == "/":
                    href = "https://apnews.com" + href 
                inner_link.append([content, href])
                para_ele = para.name

            article_paragraphs.append([index, inner_text, inner_link])
        article_information["mainContents"] = article_paragraphs
    except:
        print("no paragraphs")
    
    master_list_of_articles.append(article_information)
    # print(master_list_of_articles)
    JSONifiedArticleList = json.dumps(master_list_of_articles, ensure_ascii=False)
    with open(filename, mode='a', encoding='utf-8') as file:
        file.write(JSONifiedArticleList) 
    # print(JSONifiedArticleList)
    
# save_list_to_file(JSONifiedArticleList, filename) 

In [53]:
# Main function to update the master_list_of_articles list

#print(APArticlesList[0][1])

for i, ele in enumerate(APArticlesList):
    print(i, len(ele[1]))
    if i < 10:
    
        findURL = APArticlesList[i][1]
        print(findURL)

        article_info_function(findURL)
        time.sleep(0.2)
    else:
        pass

    
print(JSONifiedArticleList)
# Place the master_list_of_articles into a master file
        
#with open(filename, mode='w', encoding='utf-8') as file:
    #file.write(JSONifiedArticleList) 

0 2
https://apnews.com/article/video-cop-hood-chase-iowa-arrest-cbdc638ee621f08d1f6e2b99997e7523
1 2
https://apnews.com/article/taser-95-year-old-woman-australia-police-af221fd0e36e508ad8d25409ebbd8337
2 2
https://apnews.com/article/boy-slingshot-saves-sister-e1617c360273d305d18025fb6e4b9383
3 2
https://apnews.com/article/everest-british-climber-new-record-7ccfb9fcebc08771ae45a4446d420da1
4 2
https://apnews.com/article/dave-matthews-band-moon-tour-964072bbcda893ad11c28e6fb048aade
5 2
https://apnews.com/article/taser-95-year-old-woman-australia-police-af221fd0e36e508ad8d25409ebbd8337
6 2
https://apnews.com/article/boy-slingshot-saves-sister-e1617c360273d305d18025fb6e4b9383
7 2
https://apnews.com/article/everest-british-climber-new-record-7ccfb9fcebc08771ae45a4446d420da1
8 2
https://apnews.com/article/dave-matthews-band-moon-tour-964072bbcda893ad11c28e6fb048aade
9 2
https://apnews.com/article/taser-95-year-old-woman-australia-police-af221fd0e36e508ad8d25409ebbd8337
10 2
11 2
12 2
13 2
14

In [46]:
print(JSONifiedArticleList)

[]


In [45]:
with open(filename, mode='w', encoding='utf-8') as file:
    file.write(JSONifiedArticleList)
    file.close

TypeError: write() argument must be str, not list

In [24]:
print(master_list_of_articles[1])

{'self_URL': 'https://apnews.com/article/taser-95-year-old-woman-australia-police-af221fd0e36e508ad8d25409ebbd8337', 'headline': 'Australian police use Taser on 95-year-old with dementia who held steak knife', 'author(s)': ['Rod Mcguirk'], 'mainContents': [[0, 'CANBERRA, Australia (AP) — Police shocked a 95-year-old woman with a stun gun as she approached them using a walker and carrying a steak knife in an Australian nursing home, sending her to the hospital in critical condition after her head hit the floor.', []], [1, 'The extraordinary police takedown of Clare Nowland, who has dementia, on Wednesday prompted a high-level police internal investigation.', []], [2, 'It also has sparked debate about how New South Wales state police use stun guns, which are widely known as Tasers after a major manufacturer. They are a less lethal option than firearms but have occasionally proved more dangerous than other policing options.', []], [3, 'Police said Nowland received her injuries from striki

In [2]:
def load_list_from_file(filename):
    with open(filename, 'rb') as file:
        data_list = pickle.load(file)
    return data_list

APArticlesContent = list(load_list_from_file("20230519_193021-MasterListOfArticles"))

In [4]:
print(APArticlesContent)

['[', ']']
